<a href="https://colab.research.google.com/github/hetkothari/NYC-Taxi-Trip-Capstone-/blob/main/NYC_Taxi_Trip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Description
Your task is to build a model that predicts the total ride duration of taxi trips in New York City. Your primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and several other variables.

In [2]:
#Importing the libraries which will be required for this project.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV


#Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
data= pd.read_csv("/content/drive/MyDrive/NYC Taxi Data.csv")

Mounted at /content/drive


#Data Overview

In [5]:
data.head(10)

id  vendor_id  ... store_and_fwd_flag trip_duration
0  id2875421          2  ...                  N           455
1  id2377394          1  ...                  N           663
2  id3858529          2  ...                  N          2124
3  id3504673          2  ...                  N           429
4  id2181028          2  ...                  N           435
5  id0801584          2  ...                  N           443
6  id1813257          1  ...                  N           341
7  id1324603          2  ...                  N          1551
8  id1301050          1  ...                  N           255
9  id0012891          2  ...                  N          1225

[10 rows x 11 columns]

In [9]:
data.describe()

vendor_id  passenger_count  ...  dropoff_latitude  trip_duration
count  1.458644e+06     1.458644e+06  ...      1.458644e+06   1.458644e+06
mean   1.534950e+00     1.664530e+00  ...      4.075180e+01   9.594923e+02
std    4.987772e-01     1.314242e+00  ...      3.589056e-02   5.237432e+03
min    1.000000e+00     0.000000e+00  ...      3.218114e+01   1.000000e+00
25%    1.000000e+00     1.000000e+00  ...      4.073588e+01   3.970000e+02
50%    2.000000e+00     1.000000e+00  ...      4.075452e+01   6.620000e+02
75%    2.000000e+00     2.000000e+00  ...      4.076981e+01   1.075000e+03
max    2.000000e+00     9.000000e+00  ...      4.392103e+01   3.526282e+06

[8 rows x 7 columns]

In [7]:
data.info

<bound method DataFrame.info of                 id  vendor_id  ... store_and_fwd_flag trip_duration
0        id2875421          2  ...                  N           455
1        id2377394          1  ...                  N           663
2        id3858529          2  ...                  N          2124
3        id3504673          2  ...                  N           429
4        id2181028          2  ...                  N           435
...            ...        ...  ...                ...           ...
1458639  id2376096          2  ...                  N           778
1458640  id1049543          1  ...                  N           655
1458641  id2304944          2  ...                  N           764
1458642  id2714485          1  ...                  N           373
1458643  id1209952          1  ...                  N           198

[1458644 rows x 11 columns]>

In [11]:
data["trip_duration"].describe()

count    1.458644e+06
mean     9.594923e+02
std      5.237432e+03
min      1.000000e+00
25%      3.970000e+02
50%      6.620000e+02
75%      1.075000e+03
max      3.526282e+06
Name: trip_duration, dtype: float64

In [15]:
data.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dtype: object

In [16]:
data.isnull().sum()


id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [18]:
data['pickup'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff'] = pd.to_datetime(data['dropoff_datetime'])

In [26]:
data = data.drop(["pickup_datetime"],axis=1)
data = data.drop(["dropoff_datetime"],axis=1)

In [27]:
data

id  vendor_id  ...              pickup             dropoff
0        id2875421          2  ... 2016-03-14 17:24:55 2016-03-14 17:32:30
1        id2377394          1  ... 2016-06-12 00:43:35 2016-06-12 00:54:38
2        id3858529          2  ... 2016-01-19 11:35:24 2016-01-19 12:10:48
3        id3504673          2  ... 2016-04-06 19:32:31 2016-04-06 19:39:40
4        id2181028          2  ... 2016-03-26 13:30:55 2016-03-26 13:38:10
...            ...        ...  ...                 ...                 ...
1458639  id2376096          2  ... 2016-04-08 13:31:04 2016-04-08 13:44:02
1458640  id1049543          1  ... 2016-01-10 07:35:15 2016-01-10 07:46:10
1458641  id2304944          2  ... 2016-04-22 06:57:41 2016-04-22 07:10:25
1458642  id2714485          1  ... 2016-01-05 15:56:26 2016-01-05 16:02:39
1458643  id1209952          1  ... 2016-04-05 14:44:25 2016-04-05 14:47:43

[1458644 rows x 11 columns]

In [29]:
from geopy.distance import great_circle
def cal_distance(pickup_lat,pickup_long,dropoff_lat,dropoff_long):
 
 start_coordinates=(pickup_lat,pickup_long)
 stop_coordinates=(dropoff_lat,dropoff_long)
 
 return great_circle(start_coordinates,stop_coordinates).km

In [30]:
data['distance'] = data.apply(lambda x: cal_distance(x['pickup_latitude'],x['pickup_longitude'],x['dropoff_latitude'],x['dropoff_longitude'] ), axis=1)

In [32]:
data['speed'] = (data.distance/(data.trip_duration/3600))

In [33]:
dummy = pd.get_dummies(data.store_and_fwd_flag, prefix='store_and_fwd_flag')
data = pd.concat([data,dummy], axis = 1)